In [1]:
import pandas as pd
import pyodbc
from datetime import datetime, timedelta
import os
import glob
import sys

In [2]:
# last_day = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
today = datetime.today().strftime('%Y%m%d')

MA1_directionary = "U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA1/"
MA1_file = f"MA1-{today}.dat"
MA1_file_pattern = glob.glob(os.path.join(MA1_directionary,MA1_file))
MA1_pattern = MA1_file_pattern[0]
MA1_pattern = MA1_pattern.replace("\\","/")

MA2_directionary = "U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA2/"
MA2_file = f"MA2-{today}.dat"
MA2_file_pattern = glob.glob(os.path.join(MA2_directionary,MA2_file))
MA2_pattern = MA2_file_pattern[0]
MA2_pattern = MA2_pattern.replace("\\","/")

MA3_directionary = "U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA3/"
MA3_file = f"MA3-{today}.dat"
MA3_file_pattern = glob.glob(os.path.join(MA3_directionary,MA3_file))
MA3_pattern = MA3_file_pattern[0]
MA3_pattern = MA3_pattern.replace("\\","/")

MA5_directionary = "U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA5/"
MA5_file = f"MA5-{today}.dat"
MA5_file_pattern = glob.glob(os.path.join(MA5_directionary,MA5_file))
MA5_pattern = MA5_file_pattern[0]
MA5_pattern = MA5_pattern.replace("\\","/")

In [3]:
MA1_pattern

'U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA1/MA1-20250327.dat'

In [4]:
MA2_pattern

'U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA2/MA2-20250327.dat'

In [5]:
MA3_pattern

'U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA3/MA3-20250327.dat'

In [6]:
MA5_pattern

'U:/【指數投資部】/證交所造市商報價/ETF_DB/1.ETF 歷史資料/MA5/MA5-20250327.dat'

## `讀取MA1資料`

In [7]:
def process_MA1_ETF(file, colspecs, column_names):
    
    with open(file, "r", encoding = "utf-8") as file:
        file.seek(100)
        content = file.read()
    lines = [content[i:i+100] for i in range(0, len(content), 100)]

    data = []
    for line in lines:
        if len(line) < 100:  
            continue
        parsed_line = [line[start:end].strip() for start, end in colspecs]
        data.append(parsed_line)
    
    MA1_df = pd.DataFrame(data, columns = column_names)

    # 轉換數值類型 (9(5)V9(4) 代表小數點移動4位)
    float_columns = ["成交價格", "買進價格", "賣出價格", "漲停價格", "跌停價格"]
    for col in float_columns:
        MA1_df[col] = pd.to_numeric(MA1_df[col], errors="coerce") / 10000

    MA1_df['揭示時間'] = MA1_df['揭示日期'] + MA1_df['揭示時間']
    MA1_df = MA1_df.drop(columns=['揭示日期'])
    MA1_df['揭示時間'] = pd.to_datetime(MA1_df['揭示時間'].str[:17], format='%Y%m%d%H%M%S%f')
    MA1_df = MA1_df[['證券代號','揭示時間','撮合方式','成交註記','成交價格','買進漲跌停註記','買進價格','賣出漲跌停註記','賣出價格','漲停價格','跌停價格','填充欄位']]

    MA1_df['證券代號'] = MA1_df['證券代號'].astype(str)
    MA1_df['撮合方式'] = MA1_df['撮合方式'].astype(str)
    MA1_df['成交註記'] = MA1_df['成交註記'].astype(str)
    MA1_df['成交價格'] = MA1_df['成交價格'].astype(float)
    MA1_df['買進漲跌停註記'] = MA1_df['買進漲跌停註記'].astype(str)
    MA1_df['買進價格'] = MA1_df['買進價格'].astype(float)
    MA1_df['賣出漲跌停註記'] = MA1_df['賣出漲跌停註記'].astype(str)
    MA1_df['賣出價格'] = MA1_df['賣出價格'].astype(float)
    MA1_df['漲停價格'] = MA1_df['漲停價格'].astype(float)
    MA1_df['跌停價格'] = MA1_df['跌停價格'].astype(float)
    MA1_df['填充欄位'] = MA1_df['填充欄位'].astype(str)

    return MA1_df

In [8]:
# 定義固定長度欄位規則
MA1_colspecs = [
    (0, 6),   # MA1-STKNO (證券代號) 6碼
    (6, 14),  # MA1-DSPTIM (揭示時間) 8碼
    (14, 15), # MA1-PRSCD (撮合方式註記) 1碼
    (15, 16), # MA1-MTH-FLAG (成交註記) 1碼
    (16, 25), # MA1-MTHPR (成交價格) 9碼 (5整數+4小數)
    (25, 26), # MA1-BUY-R-F (買進漲跌停註記) 1碼
    (26, 35), # MA1-BUYPR (買進價格) 9碼 (5整數+4小數)
    (35, 36), # MA1-SELL-R-F (賣出漲跌停註記) 1碼
    (36, 45), # MA1-SELLPR (賣出價格) 9碼 (5整數+4小數)
    (45, 53), # MA1-DSPDAT (揭示日期) 8碼
    (53, 62), # MA1-RAISING-PRICE (漲停價格) 9碼 (5整數+4小數)
    (62, 71), # MA1-FALLING-PRICE (跌停價格) 9碼 (5整數+4小數)
    (71, 100) # FILLER (填充欄位) 29碼
]
# 欄位名稱
MA1_column_names = [
    "證券代號", "揭示時間", "撮合方式", "成交註記", "成交價格",
    "買進漲跌停註記", "買進價格", "賣出漲跌停註記", "賣出價格",
    "揭示日期", "漲停價格", "跌停價格", "填充欄位"
]

In [9]:
MA1_df = process_MA1_ETF(file = MA1_pattern, colspecs = MA1_colspecs, column_names = MA1_column_names)
MA1_df

,證券代號,揭示時間,撮合方式,成交註記,成交價格,買進漲跌停註記,買進價格,賣出漲跌停註記,賣出價格,漲停價格,跌停價格,填充欄位
0,009803,2025-03-27 08:30:26.530,T,,0.00,,9.95,,9.96,10.95,8.97,
1,009803,2025-03-27 08:30:31.540,T,,0.00,,9.95,,9.96,10.95,8.97,
2,009803,2025-03-27 08:30:36.560,T,,0.00,,9.95,,9.96,10.95,8.97,
3,009803,2025-03-27 08:30:41.580,T,,0.00,,9.89,,9.96,10.95,8.97,
4,009803,2025-03-27 08:30:46.590,T,,0.00,,9.90,,9.96,10.95,8.97,
...,...,...,...,...,...,...,...,...,...,...,...,...
10217,009803,2025-03-27 13:29:37.370,T,,9.79,,9.77,,9.78,10.95,8.97,
10218,009803,2025-03-27 13:29:42.390,T,,9.79,,9.77,,9.78,10.95,8.97,
10219,009803,2025-03-27 13:29:47.410,T,,9.79,,9.77,,9.78,10.95,8.97,
10220,009803,2025-03-27 13:29:52.420,T,,9.79,,9.77,,9.78,10.95,8.97,


In [10]:
folder_path = "U:/【指數投資部】/證交所造市商報價/ETF_DB/2.ETF Excel檔案"

today_folder_path = os.path.join(folder_path, today)

# 確保 today 資料夾存在，如果不存在則創建
os.makedirs(today_folder_path, exist_ok=True)

In [11]:
# 構建完整的文件路徑
MA1_file_path = os.path.join(today_folder_path, f'MA1_{today}.xlsx')

MA1_df.to_excel(MA1_file_path, index=False)

## `讀取MA2資料`

In [12]:
def process_MA2_ETF(file, colspecs, column_names):
    
    with open(file, "r", encoding = "utf-8") as file:
        file.seek(100)
        content = file.read()
    lines = [content[i:i+100] for i in range(0, len(content), 100)]

    data = []
    for line in lines:
        if len(line) < 100:  
            continue
        parsed_line = [line[start:end].strip() for start, end in colspecs]
        data.append(parsed_line)
    
    # 建立 DataFrame
    MA2_df = pd.DataFrame(data, columns=column_names)

    # 轉換數值類型 (9(5)V9(4) 代表小數點移動4位)
    float_columns = ["委託價格"]
    for col in float_columns:
        MA2_df[col] = pd.to_numeric(MA2_df[col], errors="coerce") / 10000

    MA2_df['交易日期'] = MA2_df['交易日期'] + MA2_df['交易時間']
    MA2_df = MA2_df.drop(columns=['交易時間'])
    MA2_df['交易日期'] = pd.to_datetime(MA2_df['交易日期'].str[:17], format='%Y%m%d%H%M%S%f')
    MA2_df = MA2_df[['證券代號','交易日期','證券商代號','投資人帳號','委託書編號','委託種類','委託數量','委託價格','買賣別B/S','改前數量','價格類別','存續種類','空白']]

    MA2_df['證券代號'] = MA2_df['證券代號'].astype(str)
    MA2_df['證券商代號'] = MA2_df['證券商代號'].astype(str)
    MA2_df['投資人帳號'] = MA2_df['投資人帳號'].astype(str)
    MA2_df['委託書編號'] = MA2_df['委託書編號'].astype(str)
    MA2_df['委託種類'] = MA2_df['委託種類'].astype(str)
    MA2_df['委託數量'] = MA2_df['委託數量'].astype(float)
    MA2_df['委託價格'] = MA2_df['委託價格'].astype(float)
    MA2_df['買賣別B/S'] = MA2_df['買賣別B/S'].astype(str)
    MA2_df['改前數量'] = MA2_df['改前數量'].astype(float)
    MA2_df['價格類別'] = MA2_df['價格類別'].astype(str)
    MA2_df['存續種類'] = MA2_df['存續種類'].astype(str)
    MA2_df['空白'] = MA2_df['空白'].astype(str)

    return MA2_df

In [13]:
MA2_colspecs = [
    (0, 6),    # MA2-STKNO      X(06)      證券代號
    (6, 10),   # MA2-BRKID      X(04)      證券商代號
    (10, 17),  # MA2-IVACNO     9(07)      投資人帳號
    (17, 22),  # MA2-ODRNO      X(05)      委託書編號
    (22, 23),  # MA2-TXCD       9(01)      委託種類
    (23, 31),  # MA2-ODRQTY     9(08)      委託數量
    (31, 40),  # MA2-ODRPR      9(5)V9(4)  委託價格
    (40, 41),  # MA2-BUYSELL    X(01)      買賣別B/S
    (41, 49),  # MA2-TXDAT      9(08)      交易日期
    (49, 57),  # MA2-TXTIME     9(08)      交易時間
    (57, 65),  # MA2-BFQTY      9(08)      改前數量
    (65, 66),  # MA2-PRICE-TYPE X(01)      價格類別
    (66, 67),  # MA2-DURATION-TYPE X(01)   存續種類
    (67, 100)  # FILLER         X(33)      空白
]

MA2_column_names = [
    "證券代號", "證券商代號", "投資人帳號", "委託書編號", "委託種類",
    "委託數量", "委託價格", "買賣別B/S", "交易日期",
    "交易時間", "改前數量", "價格類別", "存續種類",'空白'
]

In [14]:
MA2_df = process_MA2_ETF(file = MA2_pattern, colspecs = MA2_colspecs, column_names = MA2_column_names)
MA2_df

,證券代號,交易日期,證券商代號,投資人帳號,委託書編號,委託種類,委託數量,委託價格,買賣別B/S,改前數量,價格類別,存續種類,空白
0,009803,2025-03-27 09:02:48.690,592T,7777777,f006x,4,100000.0,9.89,S,100000.0,2,0,
1,009803,2025-03-27 09:02:48.690,592T,7777777,f007C,2,300000.0,9.84,S,0.0,2,0,
2,009803,2025-03-27 09:02:47.930,592T,7777777,f006t,2,300000.0,9.85,S,0.0,2,0,
3,009803,2025-03-27 09:02:47.930,592T,7777777,f006u,2,100000.0,9.86,S,0.0,2,0,
4,009803,2025-03-27 09:02:47.930,592T,7777777,f006v,2,100000.0,9.87,S,0.0,2,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4382,009803,2025-03-27 13:26:37.230,9A0T,7777777,C0S4a,2,100000.0,9.86,S,0.0,2,0,
4383,009803,2025-03-27 13:26:37.230,9A0T,7777777,C0S4l,2,100000.0,9.85,S,0.0,2,0,
4384,009803,2025-03-27 13:26:52.260,9A0T,7777777,C0S6N,4,100000.0,9.87,S,100000.0,2,0,
4385,009803,2025-03-27 13:27:17.340,9A0T,7777777,C0S8d,4,100000.0,9.88,S,100000.0,2,0,


In [15]:
# 構建完整的文件路徑
MA2_file_path = os.path.join(today_folder_path, f'MA2_{today}.xlsx')

# 將 DataFrame 保存到指定文件路徑
MA2_df.to_excel(MA2_file_path, index=False)

## `讀取MA3資料`

In [16]:
def process_MA3_ETF(file, colspecs, column_names):
    
    with open(file, "r", encoding = "utf-8") as file:
        file.seek(100)
        content = file.read()
    lines = [content[i:i+100] for i in range(0, len(content), 100)]

    data = []
    for line in lines:
        if len(line) < 100:  
            continue
        parsed_line = [line[start:end].strip() for start, end in colspecs]
        data.append(parsed_line)
    
    # 建立 DataFrame
    MA3_df = pd.DataFrame(data, columns=column_names)

    # 轉換數值類型 (9(5)V9(4) 代表小數點移動4位)

    MA3_df['成交數量'] = pd.to_numeric(MA3_df['成交數量'], errors="coerce")
    float_columns = ["成交價格"]
    for col in float_columns:
        MA3_df[col] = pd.to_numeric(MA3_df[col], errors="coerce") / 10000

    MA3_df['成交時間'] = MA3_df['成交日期'] + MA3_df['成交時間']
    MA3_df = MA3_df.drop(columns=['成交日期'])
    MA3_df['成交時間'] = pd.to_datetime(MA3_df['成交時間'].str[:17], format='%Y%m%d%H%M%S%f')
    MA3_df = MA3_df[['證券代號','成交時間','成交數量','成交價格','證商代號','投資人帳號','委託種類代號','買賣別B/S','委託書編號','空白']]

    MA3_df['證券代號'] = MA3_df['證券代號'].astype(str)
    MA3_df['成交數量'] = MA3_df['成交數量'].astype(float)
    MA3_df['成交價格'] = MA3_df['成交價格'].astype(float)
    MA3_df['證商代號'] = MA3_df['證商代號'].astype(str)
    MA3_df['投資人帳號'] = MA3_df['投資人帳號'].astype(str)
    MA3_df['委託種類代號'] = MA3_df['委託種類代號'].astype(str)
    MA3_df['買賣別B/S'] = MA3_df['買賣別B/S'].astype(str)
    MA3_df['委託書編號'] = MA3_df['委託書編號'].astype(str)
    MA3_df['空白'] = MA3_df['空白'].astype(str)

    return MA3_df

In [17]:
MA3_colspecs = [
    (0, 6),    # MA3-STKNO      X(06)      證券代號
    (6, 14),   # MA3-MTHQTY     9(08)      成交數量
    (14, 23),  # MA3-MTHPR      9(5)V9(4)  成交價格
    (23, 31),  # MA3-MTHDAT     9(08)      成交日期
    (31, 39),  # MA3-MTHTIME    9(08)      成交時間
    (39, 43),  # MA3-BRKID      X(04)      證商代號
    (43, 50),  # MA3-IVACNO     9(07)      投資人帳號
    (50, 51),  # MA3-ODRTPE     9(01)      委託種類代號
    (51, 52),  # MA3-BS         X(01)      買賣別B/S
    (52, 57),  # MA3-ODRNO      X(05)      委託書編號
    (57, 100)  # FILLER         X(43)      空白
]

MA3_column_names = [
    '證券代號','成交數量','成交價格','成交日期','成交時間','證商代號','投資人帳號','委託種類代號','買賣別B/S','委託書編號','空白'
]

In [18]:
MA3_df = process_MA3_ETF(file = MA3_pattern, colspecs = MA3_colspecs, column_names = MA3_column_names)
MA3_df

,證券代號,成交時間,成交數量,成交價格,證商代號,投資人帳號,委託種類代號,買賣別B/S,委託書編號,空白
0,009803,2025-03-27 09:02:06.460,18000.0,9.83,980T,7777777,0,S,az01j,
1,009803,2025-03-27 09:02:06.460,2000.0,9.83,980T,7777777,0,S,az01j,
2,009803,2025-03-27 09:02:06.460,3000.0,9.83,980T,7777777,0,S,az01j,
3,009803,2025-03-27 09:02:06.460,5000.0,9.83,980T,7777777,0,S,az01j,
4,009803,2025-03-27 09:02:06.460,1000.0,9.83,980T,7777777,0,S,az01j,
...,...,...,...,...,...,...,...,...,...,...
1277,009803,2025-03-27 13:30:00.000,5000.0,9.78,616T,7777777,0,S,4R774,
1278,009803,2025-03-27 13:30:00.000,1000.0,9.78,616T,7777777,0,S,4R774,
1279,009803,2025-03-27 14:30:00.000,22.0,9.82,960T,7777777,0,S,QEsd5,
1280,009803,2025-03-27 14:30:00.000,122.0,9.82,960T,7777777,0,S,QEsd5,


In [19]:
Securities_Brokerage = pd.read_excel('U:/【指數投資部】/證交所造市商報價/ETF_DB/2.ETF Excel檔案/證券商基本資料.xlsx')
Securities_Brokerage = Securities_Brokerage.iloc[:, :-3]
Securities_Brokerage['證券商代號'] =Securities_Brokerage['證券商代號'].astype(str)
Securities_Brokerage['證券商名稱'] =Securities_Brokerage['證券商名稱'].astype(str)
print(Securities_Brokerage['證券商名稱'].dtypes)
Securities_Brokerage

object


,證券商代號,證券商名稱
0,1020,合庫
1,1021,合庫- 台中
2,1022,合庫-台南
3,1023,合庫-高雄
4,1024,合庫-嘉義
...,...,...
837,9A9j,永豐金-嘉義
838,9A9q,永豐金-潮州
839,9A9r,永豐金-北高雄
840,9A9s,永豐金-彰化


In [20]:
MA3_df = MA3_df.merge(Securities_Brokerage, left_on='證商代號', right_on='證券商代號', how='left')
MA3_df = MA3_df.drop(columns=['證券商代號'])
MA3_df = MA3_df[['證券代號','成交時間','成交數量','成交價格','證商代號','證券商名稱','投資人帳號','委託種類代號','買賣別B/S','委託書編號','空白']]
MA3_df['證券商名稱'] = MA3_df['證券商名稱'].astype(str)

MA3_df

,證券代號,成交時間,成交數量,成交價格,證商代號,證券商名稱,投資人帳號,委託種類代號,買賣別B/S,委託書編號,空白
0,009803,2025-03-27 09:02:06.460,18000.0,9.83,980T,元大,7777777,0,S,az01j,
1,009803,2025-03-27 09:02:06.460,2000.0,9.83,980T,元大,7777777,0,S,az01j,
2,009803,2025-03-27 09:02:06.460,3000.0,9.83,980T,元大,7777777,0,S,az01j,
3,009803,2025-03-27 09:02:06.460,5000.0,9.83,980T,元大,7777777,0,S,az01j,
4,009803,2025-03-27 09:02:06.460,1000.0,9.83,980T,元大,7777777,0,S,az01j,
...,...,...,...,...,...,...,...,...,...,...,...
1277,009803,2025-03-27 13:30:00.000,5000.0,9.78,616T,中國信託,7777777,0,S,4R774,
1278,009803,2025-03-27 13:30:00.000,1000.0,9.78,616T,中國信託,7777777,0,S,4R774,
1279,009803,2025-03-27 14:30:00.000,22.0,9.82,960T,富邦,7777777,0,S,QEsd5,
1280,009803,2025-03-27 14:30:00.000,122.0,9.82,960T,富邦,7777777,0,S,QEsd5,


In [21]:
has_nan =  (MA3_df['證券商名稱'] == "nan").any()
if has_nan:
    print("有新增造市的證券商，需要查看MA3的證券商代號，然後從證券商基本資料.xlsx這個檔案內調整。")
    # 篩選出 '證券商名稱' 列中值為 NaN 的行
    nan_rows = MA3_df[MA3_df['證券商名稱'] == "nan"]
    print("對應的證券商代號如下：")
    print(nan_rows['證商代號'].values)
    sys.exit("程式已停止，請根據列印出來的證券商代號，更新完excel內的證券商代號後再重新執行程式碼_目前進度不會影響資料庫。")
else:
    print("資料成功合併")

資料成功合併


In [22]:
# 構建完整的文件路徑
MA3_file_path = os.path.join(today_folder_path, f'MA3_{today}.xlsx')

# 將 DataFrame 保存到指定文件路徑
MA3_df.to_excel(MA3_file_path, index=False)

## `讀取MA5資料`

In [23]:
def process_MA5_ETF(file, colspecs, column_names):
    
    with open(file, "r", encoding = "utf-8") as file:
        file.seek(100)
        content = file.read()
    lines = [content[i:i+100] for i in range(0, len(content), 100)]

    data = []
    for line in lines:
        if len(line) < 100:  
            continue
        parsed_line = [line[start:end].strip() for start, end in colspecs]
        data.append(parsed_line)
    
    # 建立 DataFrame
    MA5_df = pd.DataFrame(data, columns=column_names)

    # 轉換數值類型 (9(5)V9(4) 代表小數點移動4位)

    # 建立 DataFrame
    MA5_df = pd.DataFrame(data, columns=column_names)

    MA5_df['最佳一檔買賣價差>1%且>30分鐘次數'] = pd.to_numeric(MA5_df['最佳一檔買賣價差>1%且>30分鐘次數'], errors="coerce")
    MA5_df['單邊揭示逾3分鐘次數'] = pd.to_numeric(MA5_df['單邊揭示逾3分鐘次數'], errors="coerce")
    MA5_df['瞬間延緩撮合申報買賣<8張次數'] = pd.to_numeric(MA5_df['瞬間延緩撮合申報買賣<8張次數'], errors="coerce")

    float_columns = ["日週轉率",'市場股票日週轉率']
    for col in float_columns:
        MA5_df[col] = pd.to_numeric(MA5_df[col], errors="coerce") / 100

    MA5_df['成交日期'] = pd.to_datetime(MA5_df['成交日期'],format='%Y%m%d') 
    MA5_df = MA5_df[['證券代號','成交日期','價差超過次數註記','最佳一檔買賣價差>1%且>30分鐘次數','日週轉率<市場股票日週轉率註記',
                     '日週轉率','市場股票日週轉率','揭示超過次數註記','單邊揭示逾3分鐘次數','超過 0 次註記','瞬間延緩撮合申報買賣<8張次數',
                     '空白']]
    

    MA5_df['證券代號'] = MA5_df['證券代號'].astype(str)
    MA5_df['價差超過次數註記'] = MA5_df['價差超過次數註記'].astype(str)
    MA5_df['最佳一檔買賣價差>1%且>30分鐘次數'] = MA5_df['最佳一檔買賣價差>1%且>30分鐘次數'].astype(float)
    MA5_df['日週轉率<市場股票日週轉率註記'] = MA5_df['日週轉率<市場股票日週轉率註記'].astype(str)
    MA5_df['日週轉率'] = MA5_df['日週轉率'].astype(float)
    MA5_df['市場股票日週轉率'] = MA5_df['市場股票日週轉率'].astype(float)
    MA5_df['揭示超過次數註記'] = MA5_df['揭示超過次數註記'].astype(str)
    MA5_df['單邊揭示逾3分鐘次數'] = MA5_df['單邊揭示逾3分鐘次數'].astype(float)
    MA5_df['超過 0 次註記'] = MA5_df['超過 0 次註記'].astype(str)
    MA5_df['瞬間延緩撮合申報買賣<8張次數'] = MA5_df['瞬間延緩撮合申報買賣<8張次數'].astype(float)
    MA5_df['空白'] = MA5_df['空白'].astype(str)
    
    return MA5_df

In [24]:
# 定義固定長度欄位規則
MA5_colspecs = [
    (0, 6),    # MA5-STKNO     X(06)      證券代號
    (6, 14),   # MA5-DATE      9(08)      成交日期
    (14, 15),  # MA5-P-MARK    X(01)      超過次數註記
    (15, 23),  # MA5-P-TIMES   9(08)      最佳一檔買賣價差>1%且>30分鐘次數
    (23, 24),  # MA5-TURN-MARK X(01)      日週轉率<市場股票日週轉率註記
    (24, 30),  # MA5-TURN-RATE 9(4)V9(2)  日週轉率
    (30, 36),  # MA5-MARK-RATE 9(4)V9(2)  市場股票日週轉率
    (36, 37),  # MA5-Q-MARK    X(01)      超過次數註記
    (37, 45),  # MA5-Q-TIMES   9(08)      單邊揭示逾3分鐘次數
    (45, 46),  # MA5-R-MARK    X(01)      超過 0 次註記
    (46, 54),  # MA5-R-TIMES   9(08)      瞬間延緩撮合申報買賣<8張次數
    (54, 100)  # FILLER        X(46)      空白
]

# 欄位名稱
MA5_column_names = [
    '證券代號','成交日期','價差超過次數註記','最佳一檔買賣價差>1%且>30分鐘次數','日週轉率<市場股票日週轉率註記','日週轉率','市場股票日週轉率',
    '揭示超過次數註記','單邊揭示逾3分鐘次數','超過 0 次註記',
    '瞬間延緩撮合申報買賣<8張次數','空白'
]

In [25]:
MA5_df = process_MA5_ETF(file = MA5_pattern, colspecs = MA5_colspecs, column_names = MA5_column_names)
MA5_df

,證券代號,成交日期,價差超過次數註記,最佳一檔買賣價差>1%且>30分鐘次數,日週轉率<市場股票日週轉率註記,日週轉率,市場股票日週轉率,揭示超過次數註記,單邊揭示逾3分鐘次數,超過 0 次註記,瞬間延緩撮合申報買賣<8張次數,空白
0,009803,2025-03-27,N,0.0,N,3.27,0.35,N,0.0,N,0.0,


In [26]:
# 構建完整的文件路徑
MA5_file_path = os.path.join(today_folder_path, f'MA5_{today}.xlsx')

MA5_df.to_excel(MA5_file_path, index=False)

# `輸入資料庫內`

In [27]:
access_db_path = r'U:\【指數投資部】\證交所造市商報價\ETF_DB\ETF傳輸資料庫.accdb'

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    rf'DBQ={access_db_path};'
)
conn = pyodbc.connect(conn_str)

try:
    cursor = conn.cursor()
    for index, row in MA1_df.iterrows():
        timestamp = row['揭示時間'].strftime('%Y-%m-%d %H:%M:%S.%f')
        sql = f"INSERT INTO {'MA1_揭示檔'} ([證券代號],[揭示時間],[撮合方式],[成交註記],[成交價格],[買進漲跌停註記],[買進價格],[賣出漲跌停註記],[賣出價格],[漲停價格],[跌停價格],[填充欄位]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        cursor.execute(sql, row['證券代號'], timestamp, row['撮合方式'], row['成交註記'], row['成交價格'], row['買進漲跌停註記'], row['買進價格'], row['賣出漲跌停註記'], row['賣出價格'], row['漲停價格'], row['跌停價格'], row['填充欄位'])
    conn.commit()
    print("MA1 inserted successfully!")
except Exception as e:
    print(f"Error inserting MA1 data: {e}")


try:
    cursor = conn.cursor()
    for index, row in MA2_df.iterrows():
        timestamp = row['交易日期'].strftime('%Y-%m-%d %H:%M:%S.%f')
        sql = f"INSERT INTO {'MA2_委託檔'} ([證券代號], [交易日期], [證券商代號], [投資人帳號], [委託書編號], [委託種類], [委託數量], [委託價格], [買賣別B/S], [改前數量], [價格類別], [存續種類], [空白]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        cursor.execute(sql, row['證券代號'], timestamp, row['證券商代號'], row['投資人帳號'], row['委託書編號'], row['委託種類'], row['委託數量'], row['委託價格'], row['買賣別B/S'], row['改前數量'], row['價格類別'], row['存續種類'], row['空白'])
    conn.commit()
    print("MA2 inserted successfully!")
except Exception as e:
    print(f"Error inserting MA2 data: {e}")


try:
    cursor = conn.cursor()
    for index, row in MA3_df.iterrows():
        timestamp = row['成交時間'].strftime('%Y-%m-%d %H:%M:%S.%f')
        sql = f"INSERT INTO {'MA3_成交檔'} ([證券代號],[成交時間],[成交數量],[成交價格],[證商代號],[證券商名稱],[投資人帳號],[委託種類代號],[買賣別B/S],[委託書編號],[空白]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        cursor.execute(sql, row['證券代號'], timestamp, row['成交數量'], row['成交價格'], row['證商代號'], row['證券商名稱'], row['投資人帳號'], row['委託種類代號'], row['買賣別B/S'], row['委託書編號'], row['空白'])
    conn.commit()
    print("MA3 inserted successfully!")
except Exception as e:
    print(f"Error inserting MA3 data: {e}")


try:
    cursor = conn.cursor()
    for index, row in MA5_df.iterrows():
        timestamp = row['成交日期'].strftime('%Y-%m-%d %H:%M:%S.%f')
        sql = f"INSERT INTO {'MA5_績效標準檔'} ([證券代號],[成交日期],[價差超過次數註記],[最佳一檔買賣價差>1%且>30分鐘次數],[日週轉率<市場股票日週轉率註記],[日週轉率],[市場股票日週轉率],[揭示超過次數註記],[單邊揭示逾3分鐘次數],[超過 0 次註記],[瞬間延緩撮合申報買賣<8張次數],[空白]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        cursor.execute(sql, row['證券代號'], timestamp, row['價差超過次數註記'], row['最佳一檔買賣價差>1%且>30分鐘次數'], row['日週轉率<市場股票日週轉率註記'], row['日週轉率'], row['市場股票日週轉率'], row['揭示超過次數註記'], row['單邊揭示逾3分鐘次數'], row['超過 0 次註記'], row['瞬間延緩撮合申報買賣<8張次數'], row['空白'])
    conn.commit()
    print("MA5 inserted successfully!")
except Exception as e:
    print(f"Error inserting MA5 data: {e}")

finally:
    conn.close()

MA1 inserted successfully!
MA2 inserted successfully!
MA3 inserted successfully!
MA5 inserted successfully!
